<a href="https://colab.research.google.com/github/Hanhpt23/SmallGPT/blob/main/SmallGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building small GPT
source: [GPT](https://www.youtube.com/watch?v=kCc8FmEb1nY&t=19s&ab_channel=AndrejKarpathy)

In [58]:
import torch
import torch.nn as nn
hidden_states = torch.rand(1, 20, 512)

new_shape = hidden_states.size()[:-1] + (8, 64)
new_shape
hidden_states = hidden_states.view(new_shape)
hidden_states.permute(0, 2, 1, 3).shape

torch.Size([1, 8, 20, 64])

# Data and data transform

In [61]:
!git clone https://github.com/Hanhpt23/SmallGPT

Cloning into 'SmallGPT'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 18 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (18/18), 6.59 KiB | 6.59 MiB/s, done.


In [62]:
with open('SmallGPT/source/TheRaven.txt', 'r', encoding= 'utf-8') as f:
  text = f.read()

print("length of dataset in characters: ", len(text))


length of dataset in characters:  6648


In [63]:
print(text[:100])


Once upon a midnight dreary, while I pondered, weak and weary,
Over many a quaint and curious volum


In [64]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 1000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# Download from https://github.com/Hanhpt23/SmallGPT/source/TheRaven.txt
with open('SmallGPT/source/TheRaven.txt', 'r', encoding= 'utf-8') as f:
  text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)


In [65]:
print(chars[:40])
vocab_size

['\n', ' ', '!', ',', '-', '.', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'W', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm']


57

## We decoder the poem into integers, in which every character is mapped by a number

In [68]:
# create a mapping from characters to integers

stoi = { ch:i for i,ch in enumerate(chars) } # map characters to integers [A B C D ...] ---> [1 2 3 4 ...]
itos = { i:ch for i,ch in enumerate(chars) } # map integers to characters [1 2 3 4 ...] ---> [A B C D ...]

encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string



In [69]:
# Test the encode and decode

print(encode('Hi, how can I help you'))
print(decode(encode('Hi, how can I help you')))

[15, 35, 3, 1, 34, 41, 49, 1, 29, 27, 40, 1, 16, 1, 34, 31, 38, 42, 1, 51, 41, 47]
Hi, how can I help you


In [70]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
print(data.shape,data.dtype,  train_data.shape, val_data.shape)

torch.Size([6648]) torch.int64 torch.Size([5983]) torch.Size([665])


In [71]:
block_size = 8
print(train_data[: block_size +1],'\n--------------------------')

x = train_data[: block_size]
y = train_data[1:block_size +1]
for i in range(block_size):
  context = x[: i+1]
  target = y[i]
  print(f'When input is {context}, the target is {target}')

tensor([ 0, 20, 40, 29, 31,  1, 47, 42, 41]) 
--------------------------
When input is tensor([0]), the target is 20
When input is tensor([ 0, 20]), the target is 40
When input is tensor([ 0, 20, 40]), the target is 29
When input is tensor([ 0, 20, 40, 29]), the target is 31
When input is tensor([ 0, 20, 40, 29, 31]), the target is 1
When input is tensor([ 0, 20, 40, 29, 31,  1]), the target is 47
When input is tensor([ 0, 20, 40, 29, 31,  1, 47]), the target is 42
When input is tensor([ 0, 20, 40, 29, 31,  1, 47, 42]), the target is 41


Each number represents for a unique character
<br>===> The model should predict the next character from the given characters

## Split data into train and val sets

In [72]:
len(data) - block_size
ix = torch.randint(len(data) - block_size, (batch_size,))
x = torch.stack([data[i:i+block_size] for i in ix])
x


tensor([[31, 39, 28, 31, 44,  1, 35, 46],
        [38, 31, 27, 30,  7, 52, 46, 31],
        [31,  1, 23, 27, 48, 31, 40,  1],
        [41, 38, 38, 41, 49, 31, 30,  1],
        [31,  1, 41, 40, 38, 51,  1, 49],
        [27, 35, 40, 46, 38, 51,  1, 51],
        [31, 39, 28, 31, 44,  1, 35, 46],
        [31, 45, 41, 38, 27, 46, 31,  1],
        [45, 41,  1, 27, 42, 46, 38, 51],
        [ 1, 44, 27, 44, 31,  1, 27, 40],
        [ 1, 27, 33, 27, 35, 40,  1, 16],
        [41,  1, 28, 38, 27, 29, 37,  1],
        [31, 40, 31, 45, 45,  1, 16,  1],
        [ 1, 35, 40, 46, 41,  1, 46, 34],
        [ 1, 45, 39, 35, 38, 35, 40, 33],
        [35, 48, 35, 40, 33,  1, 34, 47]])

In [73]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # a batch dataset taken from train_data or val_dataset
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


x, y = get_batch('train')

print(len(x), '\nThe last chunk of train dataset')
print(x, '\n', y)

16 
The last chunk of train dataset
tensor([[27, 46,  1, 14, 41, 30,  1, 49],
        [41, 44, 52,  0, 24, 41, 39, 31],
        [45, 27, 46,  1, 30, 35, 48, 35],
        [16,  1, 39, 27, 44, 48, 31, 38],
        [ 1, 34, 31,  6,  0,  1,  1,  1],
        [28, 41, 48, 31,  1, 39, 51,  1],
        [39, 51,  1, 28, 41, 41, 37, 45],
        [34, 31, 38, 42,  1, 27, 33, 44],
        [33, 52, 38, 35, 46, 46, 38, 31],
        [46, 38, 31, 45, 45,  3, 56,  1],
        [ 1, 17, 31, 46,  1, 39, 31,  1],
        [54, 45,  1, 29, 41, 44, 31,  6],
        [46, 27, 42, 42, 35, 40, 33,  3],
        [39, 42, 31, 45, 46,  1, 46, 41],
        [38,  1, 39, 51,  1, 32, 27, 40],
        [45, 31,  1, 27, 40, 33, 31, 38]], device='cuda:0') 
 tensor([[46,  1, 14, 41, 30,  1, 49, 31],
        [44, 52,  0, 24, 41, 39, 31,  1],
        [27, 46,  1, 30, 35, 48, 35, 40],
        [ 1, 39, 27, 44, 48, 31, 38, 38],
        [34, 31,  6,  0,  1,  1,  1,  1],
        [41, 48, 31,  1, 39, 51,  1, 29],
        [51,  1, 28,

# Buiding a BigramLanguageModel

In [74]:

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.207161 M parameters


## Train model

In [75]:
max_iters = 5000
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.2516, val loss 4.2184
step 100: train loss 2.5017, val loss 2.5407
step 200: train loss 2.2147, val loss 2.2792
step 300: train loss 2.0481, val loss 2.1061
step 400: train loss 1.9827, val loss 2.0596
step 500: train loss 1.8756, val loss 2.0046
step 600: train loss 1.7845, val loss 1.9343
step 700: train loss 1.7150, val loss 1.8532
step 800: train loss 1.6409, val loss 1.8742
step 900: train loss 1.5704, val loss 1.8405
step 1000: train loss 1.5297, val loss 1.8028
step 1100: train loss 1.4644, val loss 1.7765
step 1200: train loss 1.4408, val loss 1.8153
step 1300: train loss 1.4118, val loss 1.7990
step 1400: train loss 1.3269, val loss 1.7808
step 1500: train loss 1.3114, val loss 1.7748
step 1600: train loss 1.2779, val loss 1.7956
step 1700: train loss 1.2512, val loss 1.8304
step 1800: train loss 1.2270, val loss 1.7991
step 1900: train loss 1.2060, val loss 1.8923
step 2000: train loss 1.1607, val loss 1.8317
step 2100: train loss 1.1488, val loss 1.8061


tensor([[0]], device='cuda:0')

In [80]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=100)[0].tolist()))



    So that darkness the wind, “thy God hather then no longer,
wh manly radiant my crest broken, wi


In [81]:
a = torch.rand((4, 4))
b = torch.ones((4, 4))
torch.tril(b)

tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])